In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score


In [32]:
# 데이터 로드 및 전처리 (이전과 동일)
data = pd.read_csv('/content/drive/MyDrive/sojong2/results.csv')
if 'total_pods' not in data.columns:
    data['total_pods'] = data['NUM_POD_A']

# 지수 감쇠 모델 함수 정의 (이전과 동일)
def exponential_decay(x, a, b, c):
    return a * np.exp(-b * (x-2)) + c

# 고유한 bandwidth 값 추출 및 정렬
bandwidth_values = np.sort(data['BANDWIDTH'].unique())

# 모델 파라미터를 저장할 딕셔너리 초기화
model_params = {}

# 각 bandwidth에 대해 모델 학습 (이전과 동일)
for bw in bandwidth_values:
    bw_data = data[data['BANDWIDTH'] == bw]
    x = bw_data['total_pods'].values
    y = bw_data['end_time'].values
    if len(x) < 3:
        continue
    initial_guess = [max(y), 0.1, min(y)]
    try:
        popt, _ = curve_fit(exponential_decay, x, y, p0=initial_guess)
        model_params[bw] = popt  # popt는 [a, b, c]
    except RuntimeError:
        continue


# model_params = {1: [-25.0875, -0.0001, 32.8687],
#                 2: [0.3491, 1.6066, 4.9107],
#                 3: [0.4010, 0.8769, 4.4000],
#                 4: [0.8060, 1.2396, 4.0102],
#                 5: [0.8654, 1.2196, 3.6781],
#                 6: [0.9379, 0.8430, 3.6556],
#                 7: [0.9147, 0.4671, 3.4091],
#                 8: [0.8410, 0.4228, 3.4200]}

# 강화학습 환경 클래스 수정된 버전
class PodScalingEnv:
    def __init__(self, bandwidth_values, pod_range, model_params, alpha=0.5, k_time=1.0, k_pods=1.0):
        self.bandwidth_values = bandwidth_values
        self.pod_range = pod_range
        self.model_params = model_params
        self.alpha = alpha  # 추론 시간의 중요도
        self.k_time = k_time  # 추론 시간 스케일링 상수
        self.k_pods = k_pods  # 파드 개수 스케일링 상수
        self.state = None
        self.action_space = len(pod_range)
        self.state_space = len(bandwidth_values)

    def reset(self):
        self.state = np.random.choice(self.bandwidth_values)
        return self.state

    def step(self, action_index):
        pod_count = self.pod_range[action_index]
        if self.state in self.model_params:
            a, b, c = self.model_params[self.state]
            predicted_time = a * np.exp(-b * (pod_count-2)) + c
            # 보상 함수 계산
            f_time = np.exp(self.k_time * predicted_time)
            f_pods = np.exp(self.k_pods * pod_count)
            reward = - (self.alpha * f_time + (1 - self.alpha) * f_pods)
        else:
            reward = -1000  # 모델이 없는 경우 큰 패널티 부여
        done = True
        next_state = self.state
        return next_state, reward, done, {}

# 하이퍼파라미터 설정
alpha_rl = 0.1      # 학습률
gamma = 0.9         # 할인율
epsilon = 0.1       # 탐험률
num_episodes = 30000 # 학습 에피소드 수

# 가능한 bandwidth 값과 파드 수 범위 설정
bandwidth_values = sorted(model_params.keys())
pod_range = list(range(2, 9))

# 환경 생성 (새로운 보상 함수를 사용)
env = PodScalingEnv(
    bandwidth_values=bandwidth_values,
    pod_range=pod_range,
    model_params=model_params,
    alpha=0.8,      # 추론 시간의 중요도
    k_time=0.3,    # 추론 시간 스케일링 상수
    k_pods=0.1      # 파드 개수 스케일링 상수
)

# Q-테이블 초기화
Q = np.zeros((env.state_space, env.action_space))
state_to_index = {bw: idx for idx, bw in enumerate(bandwidth_values)}

# 학습 루프
for episode in range(num_episodes):
    state = env.reset()
    state_idx = state_to_index[state]
    done = False
    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(env.action_space)
        else:
            action = np.argmax(Q[state_idx, :])
        next_state, reward, done, _ = env.step(action)
        next_state_idx = state_to_index[next_state]
        best_next_action = np.argmax(Q[next_state_idx, :])
        Q[state_idx, action] += alpha_rl * (reward + gamma * Q[next_state_idx, best_next_action] - Q[state_idx, action])
        state_idx = next_state_idx

# 학습 완료 후 정책 추출 및 결과 출력
optimal_actions = np.argmax(Q, axis=1)
optimal_pod_counts = [pod_range[action] for action in optimal_actions]

for idx, bw in enumerate(bandwidth_values):
    a, b, c = model_params[bw]
    print(f'bandwidth {bw}에서 최적의 파드 수는 {optimal_pod_counts[idx]}입니다,{a * np.exp(-b * (optimal_pod_counts[idx]-2)) + c} ')

bandwidth 1에서 최적의 파드 수는 2입니다,7.7812228101604255 
bandwidth 2에서 최적의 파드 수는 4입니다,4.924777261685548 
bandwidth 3에서 최적의 파드 수는 5입니다,4.428898494924016 
bandwidth 4에서 최적의 파드 수는 5입니다,4.02973705614345 
bandwidth 5에서 최적의 파드 수는 5입니다,3.7004031072696533 
bandwidth 6에서 최적의 파드 수는 5입니다,3.73037432926085 
bandwidth 7에서 최적의 파드 수는 6입니다,3.5502598166824013 
bandwidth 8에서 최적의 파드 수는 6입니다,3.574961287136813 
